# Put Reviews On Kinesis Data Firehose

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
firehose = boto3.Session().client(service_name='firehose', region_name=region)
kinesis_analytics = boto3.Session().client(service_name='kinesisanalytics', region_name=region)


In [2]:
%store -r firehose_name

In [3]:
print(firehose_name)

dsoaws-kinesis-data-firehose


In [4]:
%store -r firehose_arn

In [5]:
print(firehose_arn)

arn:aws:firehose:us-east-1:835319576252:deliverystream/dsoaws-kinesis-data-firehose


In [6]:
%store -r iam_role_kinesis_arn

In [7]:
print(iam_role_kinesis_arn)

arn:aws:iam::835319576252:role/DSOAWS_Kinesis


In [8]:
%store -r kinesis_data_analytics_app_name

In [9]:
print(kinesis_data_analytics_app_name)

dsoaws-kinesis-data-analytics-sql-app


In [10]:
firehoses = firehose.list_delivery_streams(DeliveryStreamType='DirectPut')
print(firehoses)

{'DeliveryStreamNames': ['bandits-exp-1', 'dsoaws-kinesis-data-firehose'], 'HasMoreDeliveryStreams': False, 'ResponseMetadata': {'RequestId': 'c179e5d1-b2d5-2bed-96bf-82870277bf50', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c179e5d1-b2d5-2bed-96bf-82870277bf50', 'x-amz-id-2': 'pNGQIPIl1l3XrPe+t/x+118Y+tHvGt/7s3b/CRaJyYsVUVoj8fbYWGJ9Hp7Q8fnCGqblHE+i46NxXtHfcmC2WQbwPdtVWCW2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '103', 'date': 'Mon, 31 Aug 2020 02:48:54 GMT'}, 'RetryAttempts': 0}}


# Download Dataset

In [11]:
!aws s3 cp 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz' ./data/

download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz to data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz


In [12]:
import csv
import pandas as pd

df = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                 delimiter='\t', 
                 quoting=csv.QUOTE_NONE,
                 compression='gzip')
df.shape

(102084, 15)

In [13]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],Digital_Software,4,0,0,N,Y,Four Stars,So far so good,2015-08-31
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,Digital_Software,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015-08-31
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],Digital_Software,1,1,2,N,Y,One Star,Please cancel.,2015-08-31
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,Digital_Software,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015-08-31
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,Digital_Software,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015-08-31


# Simulate Application Writing Records to the Stream

In [14]:
firehose_response = firehose.describe_delivery_stream(
    DeliveryStreamName=firehose_name
)

print(firehose_response)

{'DeliveryStreamDescription': {'DeliveryStreamName': 'dsoaws-kinesis-data-firehose', 'DeliveryStreamARN': 'arn:aws:firehose:us-east-1:835319576252:deliverystream/dsoaws-kinesis-data-firehose', 'DeliveryStreamStatus': 'ACTIVE', 'DeliveryStreamEncryptionConfiguration': {'Status': 'DISABLED'}, 'DeliveryStreamType': 'DirectPut', 'VersionId': '1', 'CreateTimestamp': datetime.datetime(2020, 8, 31, 2, 33, 35, 268000, tzinfo=tzlocal()), 'Destinations': [{'DestinationId': 'destinationId-000000000001', 'S3DestinationDescription': {'RoleARN': 'arn:aws:iam::835319576252:role/DSOAWS_Kinesis', 'BucketARN': 'arn:aws:s3:::sagemaker-us-east-1-835319576252', 'Prefix': 'kinesis-data-firehose', 'BufferingHints': {'SizeInMBs': 5, 'IntervalInSeconds': 300}, 'CompressionFormat': 'UNCOMPRESSED', 'EncryptionConfiguration': {'NoEncryptionConfig': 'NoEncryption'}, 'CloudWatchLoggingOptions': {'Enabled': False}}, 'ExtendedS3DestinationDescription': {'RoleARN': 'arn:aws:iam::835319576252:role/DSOAWS_Kinesis', 'Buc

# Put Records onto Firehose

In [15]:
step = 10
for start_idx in range(0, 10000, step):
    end_idx = start_idx + step - 1

    df_star_rating_and_review_body = df[['star_rating', 'review_body']][start_idx:end_idx]

    reviews_tsv = df_star_rating_and_review_body.to_csv(sep='\t',
                                                        header=None,
                                                        index=False)
    print(reviews_tsv)

    response = firehose.put_record(        
        Record={
            'Data': reviews_tsv.encode('utf-8')
        },
        DeliveryStreamName=firehose_name
    )

4	So far so good
3	Needs a little more work.....
1	Please cancel.
5	Works as Expected!
4	I've had Webroot for a few years. It expired and I decided to purchase a renewal on Amazon. I went through hell trying to uninstall the expired version in order to install the new.  I called Webroot and had their representative remote into my computer at his request. He was clueless as a bad joke and consumed 29 minutes and 57 seconds of my time forever.  He initially told me it wasn't compatible with Windows 10, but I finally managed to convince him that it is indeed compatible with Windows 10 as it was working on my computer before it expired and also I showed him a review on Amazon to convince him that it works on Windows 10. Finally, he offered to connect me with a senior consultant for over 100 dollars. I declined and told him I'd fix the issue myself. This guy was less helpful than a severed limb.  After spending some time on Google, the issue is now fixed. Webroot should just get rid of thei

5	Awesome App.
5	works great
5	No problem downloaded without a problem
5	Perfect in its simplicity and output.
4	Every thing seemed to work fine so far.<br />Of course it is tough to tell what I covered and what is issue requires you to buy the upgrade.<br /><br />More clarity on that in the alert would be good.
5	I downloaded many different webcam software for a tryout. None of the software had so much features or as easy to use as this Debut Video Capture software.
1	Worthless, doesn't work right and have no support
1	Made the mistake of trying System Mechanic (free trial) on Windows 10 even though I vaguely recall it being crap on an XP system years ago. Well it's still crap!  Went back to using TuneUp Utilities and very happy with it.
5	Simple, effective, inexpensive.

1	If you want a free audio tool use Audacity. Use Audacity. USE AUDACITY! It works great I made tons of song with it because Fl Studio 12 is hell too work with recording.
4	I like this product but it was extremely fr

5	I always jasper sky Internet security every year! Love the product! Doesn't let anything by unless told to!
1	Don't waste your money. It's really difficult to navigate. The character's voice is annoying too. I think the company should upgrade this product so that it's more up to date with other  educational video software. My grandkids did n't really enjoy it for the same reasons aforementioned.
1	coming from windows, it should be free!!!
5	This product would be great if I could ever get my username and password right.
5	My windows 10 had a hijack attempt twice. I couldn't close the browser so I unplugged myself from the network. Minimized the browser and opened this which forced the browser closed. After that, I cleaned up my system files.
2	Constant pop-ups with cryptic messages, a noticeable decrease in computer speed, scans that take over an hour, these are just some of the reasons I'm sorry I bought this. I'm sure it is lovely at keeping out malware, but compared to others like 

5	I used Corel Wordperfect for years and this is much better.
5	Great product if you want a professional resume.
1	Microsoft just can't seem to get it right.  Every time they do an &#34;upgrade&#34;, it be becomes less user friendly.  IE is constantly freezing up and for some reason, my track pad doesn't seem to operate with Windows 10.  And Edge is horrible.  Who at Microsoft thought that was a good idea?
5	It is a dynamite program and keeps my computer running great. I would recommend System Mechanic to every one. I depend on it to keep my computer junk free and it does.
5	quick and easy instant download. So far i have had no issues with my macbook so its doing the job.
3	I bought this because WebRoot said it's &#34;Multi-Device Coverage provides always-on protection for PC, Mac®, Android and iOS. I have a Mac, PC and Android and two iPhones. I assumed that once I purchased SecureAnywhere Internet Security Complete 2015, that I wouldn't be required to pay for additional platforms. Th

5	perfect
5	I have been looking for a great electronic cookbook for all of my wife's old manual file recipes and Cook'n has filled the need.  Not only is the Cook'n software easy to use when installing the recipes, but I now can carry the recipes with us when we travel.  They are on my iphone, ipad, and laptop.<br />With all of the recipes on the iphone, it makes it very convenient to shop at the grocery store and purchase everything that is need for the evening meal. Even my son, who lives 600 miles away has a copy of all the recipes on his note book.  The support team at Cook'n has been great to work with.  Their response time to any problem that arose was addressed with in 24 hours of my request. They believe strongly about their software and want to the customer to be just excited about using it.<br />Bottom line is &#34;If you want at great electronic recipe fie at your finger tips, you should really take a good look at the Cook'n program.&#34;
1	My employer just upgraded to this 

...

4	Arrived quickly and installed without problem.  I'm really not a fan of Quicken since I use Quick Books also, and really enjoy how intuitive their menus, processes, etc are.  In the case of Quicken, it seems that the conventions are entirely different and they have gone to great lengths to avoid &#34;financial mumbo jumbo&#34; and in the process of using that approach, they make it almost impossible to do things like print a simple report or do a &#34;quick report&#34; on a particular &#34;customer.&#34;  No complaint at all about the vendor here or the service; my gripe is with the software itself, which I have been using for at least a dozen years now.  I keep thinking the folks at Intuit will someday do a complete makeover to make personal finace accounting  more similar to small biz financial accounting.
5	Totally Awesome!
5	best price I found
2	Could not get it to incorporate last year's info
5	My computer has been trouble free since I purchased pc matic over a year ago. Best pr

3	They really need to NOT try to make a buck off their bread and butter. We give them a ton of money for not much work on their end. Do not remove the forms that have always been included!
1	Cannot install without registering with Microsoft. Useless if you are an IT person installing on 10 PC's as you will need 10 separate MS Accounts. Users come and go in offices and it is going to be very difficult keeping track of what is installed on which computer with which user account and which password.
1	Filed 2014 return early April. Requested $25 refund offered to users of deluxe edition. Web said could not process my request and I should call Turbotax. I do not recall any mention of a deadline. Today (May 11, 2015) I tried again for refund.It apparently has expired as of April 14, 2015. User frustration is amplified as you try to reach a live agent or sort through dead links. I have used Turbotax for 20+ years and aside from occasional minor glitches it has worked well. The 2014 experience

5	Did not have any issues loading software/deleting old one
5	the most useful tool at tax time. Great price and easy to use!
1	Ever since I downloaded this new version my computer has not been functioning properly. Some programs no longer work such as itunes and other programs. No amount of updates have any effect. Norton keeps showing a red flag on the quick scan mode. It won't complete the scan. Even the power scrubber can't find any problems. Previous versions, over the years, have performed as they should. I'm disappointed that Norton put out what seems to be an incomplete version of their security software.
2	I messed with this and still ended up using an accountant.  For some people it would be ok, but in this case I would have been better served to have left it to the professionals.
5	As good/or better than any I've paid for. I recommend it to every home user.
1	horrible, returned and just using their online software instead.
1	It is sad that it must go the web to do most functi

5	I switched to TaxCut this year when Turbo Tax did their flim flam on pricing. Filed my Fed and State taxes with no probem at half the price of TT. Handles Schedule C and Itemized Deductions flawlessly
5	As always fast and easy to use!
5	Keeping my PC tuned up and protected. good value for performance.
5	Works as described
5	I have used TurboTax for years and I am always satisfied with the ease of use and results.
2	Error-prone, too many bugs, cumbersome. I've since switched to iBank 5.x and am very happy that I did.
4	Easy to purchase and download....used it many times already
3	&#34;System Mechanic is a full-featured system maintenance program for your computer that fixes problems and keeps things running smoothly. Through this app, you can analyze your system, identify areas for improvement, and fix errors, all with just a few clicks of the mouse.&#34; (download.com)  I think that the full version would be nice to have and the free version is obviously limited but it was FREE!<br /

2	First off, purposely disabling the features [One Step Update] to cripple the software after a set period of time is just unacceptable.  Second, it would not import the previous financial transactions from the saved quicken files.  To get it to work at all with my financial institutions I had to start clean. All my &#34;bill reminders&#34; are gone. History of previous transactions are all gone.  What a pain this software has been to get working.  If you want to spend hours getting your financial software to work properly then this is your best bet.  It seems that year after year is just gets buggier and less helpful.  Too bad there is no more competition from Microsoft.  Then Intuit might have a reason to develop a better product.
5	As always, it did everthing I needed it to do.
5	I have been using it for several years after an IT Tech at my Corporation recommended it to me. I have used it on Windows XP Professional, Windows 7 and Windows 8.1 without any problems. It does exactly wha

# Analyze Stream with Kinesis Data Analytics

_Re-Run ^^ Above ^^ Cell If You See `No rows in source stream`_

![](./img/no_rows_in_source_kinesis_firehose_stream.png)

In [16]:
from IPython.core.display import display, HTML
        
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/kinesisanalytics/home?region={}#/wizard/editor?applicationName={}"> Kinesis Data Analytics App</a></b>'.format(region, kinesis_data_analytics_app_name)))


In [17]:
from IPython.core.display import display, HTML
    
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/firehose/home?region={}#/details/{}/monitoring"> Firehose</a></b>'.format(region, firehose_name)))


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();